In [4]:
import os, sys
import cv2
from keras.layers.core import *
from keras.layers import  Input,Dense,Flatten,Dropout,merge,Reshape,Conv2D,MaxPooling2D,UpSampling2D,Conv2DTranspose,ZeroPadding2D, Add
from keras.layers.normalization import BatchNormalization
from keras.models import Model,Sequential,load_model
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adadelta, RMSprop,SGD,Adam
from keras import regularizers
from keras import backend as K
import numpy as np
import scipy
import numpy.random as rng
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt

Using TensorFlow backend.


ModuleNotFoundError: No module named 'sklearn.cross_validation'

In [ ]:
def Regressor(input_img):
	#merg1 = merge([input_img, decoded], mode='concat', concat_axis=3, name = "merge_r1")
	reg_conv1_1 = Conv2D(16, (3, 3), activation='relu', padding='same', name = "block1_conv1")(merg1)
	reg_conv1_1 = BatchNormalization()(reg_conv1_1)
	reg_conv1_2 = Conv2D(16, (3, 3), activation='relu', padding='same',  name = "block1_conv2")(reg_conv1_1)
	reg_conv1_2 = BatchNormalization()(reg_conv1_2)
	reg_pool1 = MaxPooling2D(pool_size=(2, 2),strides=(2,2),padding='same', name = "block1_pool1")(reg_conv1_2)
	
	reg_conv2_1 = Conv2D(64, (3, 3), activation='relu', padding='same', name = "block2_conv1")(reg_pool1)
	reg_conv2_1 = BatchNormalization()(reg_conv2_1)
	reg_conv2_2 = Conv2D(64, (3, 3), activation='relu', padding='same', name = "block2_conv2")(reg_conv2_1)
	reg_conv2_2 = BatchNormalization()(reg_conv2_2)
	reg_pool2= MaxPooling2D(pool_size=(2, 2),strides=(2,2), padding='same', name = "block2_pool1")(reg_conv2_2)

	reg_conv3_1 = Conv2D(128, (3, 3), activation='relu', padding='same', name = "block3_conv1")(reg_pool2)
	reg_conv3_1 = BatchNormalization()(reg_conv3_1)
	reg_conv3_2 = Conv2D(128, (3, 3), activation='relu', padding='same', name = "block3_conv2")(reg_conv3_1)
	reg_conv3_2 = BatchNormalization()(reg_conv3_2)
	reg_pool3 = MaxPooling2D(pool_size=(2, 2),strides=(2,2), padding='same', name = "block3_pool1")(reg_conv3_2)
    
	reg_flat = Flatten()(reg_pool3)
	fc1 = Dense(256, activation='relu')(reg_flat)
    fc2 = Dense(64, activation='relu')(fc1)
	fc3 = Dense(16, activation='relu')(fc2)
	fc4 = Dense(2, activation='relu')(fc3)
	regress = Model(inputs = input_img, outputs =  fc4)
	return regress

In [2]:
def load_data():
    imagePath = None
    maskPath = None
    gtPath = None

    files = []
    if os.path.exists(imagePath) and os.path.exists(maskPath) and os.path.exists(gtPath):
        files = os.listdir(imagePath)
    else:
        sys.exit("Invalid Path")
    images = []
    masks = []
    gt = []
    for file in files:
        im = cv2.imread(imagePath+file,0)
        im = cv2.resize(im, (256,256))
        im = np.reshape(im, (256,256,1))
        images.append(im)

        im = cv2.imread(maskPath+file,0)
        im = cv2.resize(im, (256,256))
        im = np.reshape(im, (256,256,1))
        masks.append(im)

        f = open(gtPath+file, 'r')
        x, y = map(int(), f.readline().split())
        gt.append((x,y))
    images = np.array(images)
    masks = np.array(masks)
    y = np.array(gt)
    x = np.dstack(images,masks)
    X_train,X_test,Y_train,Y_test=train_test_split(x,y,test_size=0.001)
    return X_train, X_test, Y_train, Y_test

In [3]:
X_train, X_test, Y_train, Y_test = load_data()

NameError: name 'os' is not defined

In [ ]:
regressorInput = Input((X_train.shape[1], X_train.shape[2], X_train.shape[3]))
regressor = Regressor(regressorInput)
regressor.compile(optimizer = Adam(0.0005), loss= 'mean_squared_error')
regressor.summary()

In [ ]:
batch_size = 128
epochs = 5
regressor.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs,
              validation_data=(X_test,Y_test), shuffle=True, verbose=1) 